# ChildELECTRA
- https://github.com/monologg/KoELECTRA<br>
- https://monologg.kr/2020/05/02/koelectra-part2/<br>
- https://huggingface.co/docs/transformers/model_doc/electra

In [ ]:
##Parameter setting
setEpoch = 10
setLearningRate = 0.0001
setEpsilon = 1e-8
setBatch = 32
setMaxLength = 256
setSeed = 42
labelNumber = 2
setTry = 31

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
!pip install transformers
!pip install tensorflow
!pip install keras
!pip install sentencepiece

     |████████████████████████████████| 2.9 MB 7.0 MB/s 
     |████████████████████████████████| 895 kB 67.3 MB/s 
     |████████████████████████████████| 596 kB 74.6 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 72.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 7.7 MB/s 


In [ ]:
import tensorflow as tf
import torch

from transformers import ElectraTokenizer
from transformers import ElectraModel, ElectraForSequenceClassification, AdamW, ElectraConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import sentencepiece as spm

In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for currentTry in range(11,setTry):
  import pandas as pd

  fileDir = "drive/My Drive/2022년/문성민 박사/딥러닝/BiClassification/Data/Binary/trainBi.csv"

  fr = open(fileDir, 'r')
  contents = fr.readlines()
  fr.close()

  train = pd.DataFrame(columns=('Label', 'Sentence'))
  i = 0
  label = ""
  sentence = ""
  for content in contents:
      if i == 0:
          pass
      else:
          infos = content.split(",")
          label = int(infos[2])
          sentence = infos[3].replace("\n", "")
          train.loc[i] = [label, sentence]
      i = i + 1


  train['Sentence'] = train['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ",regex=True)
  train['Sentence'] = train['Sentence'].str.replace(r'\t+', " ", regex=True)
  train['Sentence'] = train['Sentence'].str.replace(r'[\\n]+', " ", regex=True)

  # 리뷰 문장 추출
  sentences = train['Sentence']

  # BERT의 입력 형식에 맞게 변환
  sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

  # 라벨 추출
  labels = train['Label'].values
  labels_re = []
  for label in labels:
    labels_re.append(label)
  labels = labels_re

  # # BERT의 토크나이저로 문장을 토큰으로 분리
  # from kobert_transformers import get_tokenizer
  # tokenizer = get_tokenizer()
  # tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  # BERT의 토크나이저로 문장을 토큰으로 분리
  tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]  

  # 입력 토큰의 최대 시퀀스 길이
  MAX_LEN = setMaxLength  # 64 128 256 512

  # 토큰을 숫자 인덱스로 변환
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  print(input_ids[0])


  # 어텐션 마스크 초기화
  attention_masks = []

  # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
  # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  print(attention_masks[0])
  print(labels)
  print(input_ids)


  # 훈련셋과 검증셋으로 분리
  train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                      labels, 
                                                                                      random_state=2018, 
                                                                                      test_size=0.05)

  # 어텐션 마스크를 훈련셋과 검증셋으로 분리
  train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                        input_ids,
                                                        random_state=2018, 
                                                        test_size=0.05)

  # 데이터를 파이토치의 텐서로 변환
  train_inputs = torch.tensor(train_inputs)
  train_labels = torch.tensor(train_labels)
  train_masks = torch.tensor(train_masks)
  validation_inputs = torch.tensor(validation_inputs)
  validation_labels = torch.tensor(validation_labels)
  validation_masks = torch.tensor(validation_masks)

  # 배치 사이즈
  batch_size = setBatch

  # 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
  # 학습시 배치 사이즈 만큼 데이터를 가져옴
  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  # 분류를 위한 BERT 모델 생성
  model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=labelNumbe)
  model.cuda()

  # 정확도 계산 함수
  def flat_accuracy(preds, labels):
      
      pred_flat = np.argmax(preds, axis=1).flatten()
      labels_flat = labels.flatten()

      return np.sum(pred_flat == labels_flat) / len(labels_flat)

  # 시간 표시 함수
  def format_time(elapsed):

      # 반올림
      elapsed_rounded = int(round((elapsed)))
      
      # hh:mm:ss으로 형태 변경
      return str(datetime.timedelta(seconds=elapsed_rounded))

  # 입력 데이터 변환
  def convert_input_data(sentences):

      # BERT의 토크나이저로 문장을 토큰으로 분리
      tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

      # 입력 토큰의 최대 시퀀스 길이
      MAX_LEN = 256

      # 토큰을 숫자 인덱스로 변환
      input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
      
      # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
      input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

      # 어텐션 마스크 초기화
      attention_masks = []

      # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
      # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
      for seq in input_ids:
          seq_mask = [float(i>0) for i in seq]
          attention_masks.append(seq_mask)

      # 데이터를 파이토치의 텐서로 변환
      inputs = torch.tensor(input_ids)
      masks = torch.tensor(attention_masks)

      return inputs, masks

  ##############################결과구문으로출력###################################
  def outreault(guess):
      guess = int(guess)
      outConstruction = ""
      if guess == 0:
          outConstruction = "agent-first"
      elif guess == 1:
          outConstruction = "theme-first"

      return outConstruction


  # 옵티마이저 설정
  optimizer = AdamW(model.parameters(),
                    lr = setLearningRate, # 학습률
                    eps = setEpsilon # 0으로 나누는 것을 방지하기 위한 epsilon 값
                  )

  # 에폭수
  epochs = setEpoch

  # 총 훈련 스텝 : 배치반복 횟수 * 에폭
  total_steps = len(train_dataloader) * epochs

  # 학습률을 조금씩 감소시키는 스케줄러 생성
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0,
                                              num_training_steps = total_steps)


  #학습
  # 재현을 위해 랜덤시드 고정
  seed_val = setSeed
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # 그래디언트 초기화
  model.zero_grad()


  outDir = "drive/My Drive/2022년/문성민 박사/딥러닝/BiClassification/Data/Binary_2023/OutputELECTRA/electraBi_All_T"+str(currentTry)+".csv"
  f = open(outDir, 'w')
  f.write("epoch,sentence,originalLabel,predictedLabel,predictedConstruction,result"+"\n")

  # 에폭만큼 반복
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # 시작 시간 설정
      t0 = time.time()

      # 로스 초기화
      total_loss = 0

      # 훈련모드로 변경
      model.train()
          
      # 데이터로더에서 배치만큼 반복하여 가져옴
      for step, batch in enumerate(train_dataloader):
          # 경과 정보 표시
          if step % 500 == 0 and not step == 0:
              elapsed = format_time(time.time() - t0)
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # 배치를 GPU에 넣음
          batch = tuple(t.to(device) for t in batch)
          
          # 배치에서 데이터 추출
          b_input_ids, b_input_mask, b_labels = batch

          # Forward 수행                
          outputs = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask, 
                          labels=b_labels)
          
          # 로스 구함
          loss = outputs[0]

          # 총 로스 계산
          total_loss += loss.item()

          # Backward 수행으로 그래디언트 계산
          loss.backward()

          # 그래디언트 클리핑
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # 그래디언트를 통해 가중치 파라미터 업데이트
          optimizer.step()

          # 스케줄러로 학습률 감소
          scheduler.step()

          # 그래디언트 초기화
          model.zero_grad()

      # 평균 로스 계산
      avg_train_loss = total_loss / len(train_dataloader)            

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
          
      # ========================================
      #               Validation
      # ========================================

      print("")
      print("Running Validation...")

      #시작 시간 설정
      t0 = time.time()

      # 평가모드로 변경
      model.eval()

      # 변수 초기화
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      # 데이터로더에서 배치만큼 반복하여 가져옴
      for batch in validation_dataloader:
          # 배치를 GPU에 넣음
          batch = tuple(t.to(device) for t in batch)
          
          # 배치에서 데이터 추출
          b_input_ids, b_input_mask, b_labels = batch
          
          # 그래디언트 계산 안함
          with torch.no_grad():     
              # Forward 수행
              outputs = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask)
          
          # 로스 구함
          logits = outputs[0]

          # CPU로 데이터 이동
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          
          # 출력 로짓과 라벨을 비교하여 정확도 계산
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          eval_accuracy += tmp_eval_accuracy
          nb_eval_steps += 1

          

      print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
      print("  Validation took: {:}".format(format_time(time.time() - t0)))


      if (epoch_i+1) == 1 or (epoch_i+1) == 2 or (epoch_i+1) == 3 or (epoch_i+1) == 4 or (epoch_i+1) == 5 or (epoch_i+1) == 6 or (epoch_i+1) == 7 or (epoch_i+1) == 8 or (epoch_i+1) == 9 or (epoch_i+1) == 10 :#or (epoch_i+1) == 20 or (epoch_i+1) == 30: #or (epoch_i+1) == 40 or (epoch_i+1) == 50:
        # ###############################전체 모델 성능 평가########################################

        # 문장 테스트
        def test_sentences(sentences):
            # 평가모드로 변경
            model.eval()

            # 문장을 입력 데이터로 변환
            inputs, masks = convert_input_data(sentences)

            # 데이터를 GPU에 넣음
            b_input_ids = inputs.to(device)
            b_input_mask = masks.to(device)
                    
            # 그래디언트 계산 안함
            with torch.no_grad():     
                # Forward 수행
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)

            # 로스 구함
            logits = outputs[0]

            # CPU로 데이터 이동
            logits = logits.detach().cpu().numpy()

            return logits

        import pandas as pd

        testFileDir = fileDir = "drive/My Drive/2022년/문성민 박사/딥러닝/BiClassification/Data/Binary/testBi_All.csv" #"drive/My Drive/NLP/GH/BUCLD/Data/Binary/testBi_BUCLD.csv"
        testFr = open(testFileDir, 'r')
        testContents = testFr.readlines()
        testFr.close()

        test = pd.DataFrame(columns=('Label1', 'Label2','Sentence'))
        i = 0
        for content in testContents:
            if i == 0:
                pass
            else:
                infos = content.split(",")
                label1 = int(infos[0])
                label2 = int(infos[1])
                sentence = infos[2].replace("\n", "")
                test.loc[i] = [label1, label2, sentence]
            i = i + 1

        test['Sentence'] = test['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
        test['Sentence'] = test['Sentence'].str.replace(r'\t+', " ", regex=True)
        test['Sentence'] = test['Sentence'].str.replace(r'[\\n]+', " ", regex=True)

        # 리뷰 문장 추출
        testSentences = test['Sentence']

        totalNum = 0
        correctNum = 0
        for each in range(0, len(testSentences)):
            print(test['Label1'][each + 1], test['Label2'][each + 1], test['Sentence'][each + 1])
            logits = test_sentences([test['Sentence'][each + 1]])
            guess = str(np.argmax(logits))
            if guess == str(test['Label1'][each + 1]) or guess == str(test['Label2'][each + 1]):
                print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(O)")
                f.write(str(epoch_i+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess)+ ",1" + "\n")
                correctNum = correctNum + 1
            else:
                f.write(str(epoch_i+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess) + ",0" + "\n")       
                print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(X)")
            totalNum = totalNum + 1

        print("totalNum: ", totalNum, " correctNum: ", correctNum, " accuracy: ", (correctNum/totalNum))

  f.close()



      

  print("")
  print("Training complete!")


Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

[    2 23293  4007  4070 24086  4007  4110  3412  4480  7933     3     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

...


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.15
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.98
  Validation took: 0:00:03
0 0 호랑이가 코끼리를 그려요
input:  호랑이가 코끼리를 그려요 , predict:  0 (O)
0 0 원숭이가 다람쥐를 던져요
input:  원숭이가 다람쥐를 던져요 , predict:  0 (O)
0 0 꿀벌이 다람쥐를 칠해요
input:  꿀벌이 다람쥐를 칠해요 , predict:  0 (O)
0 0 꿀벌이 나비를 안아요
input:  꿀벌이 나비를 안아요 , predict:  0 (O)
0 0 다람쥐가 호랑이를 밀어요
input:  다람쥐가 호랑이를 밀어요 , predict:  0 (O)
0 0 개구리가 원숭이를 들어요
input:  개구리가 원숭이를 들어요 , predict:  0 (O)
0 0 코끼리가 호랑이 그려요
input:  코끼리가 호랑이 그려요 , predict:  0 (O)
0 0 다람쥐가 원숭이 던져요
input:  다람쥐가 원숭이 던져요 , predict:  0 (O)
0 0 다람쥐가 꿀벌 칠해요
input:  다람쥐가 꿀벌 칠해요 , predict:  0 (O)
0 0 나비가 꿀벌 안아요
input:  나비가 꿀벌 안아요 , predict:  0 (O)
0 0 호랑이가 다람쥐 밀어요
input:  호랑이가 다람쥐 밀어요 , predict:  0 (O)
0 0 원숭이가 개구리 들어요
input:  원숭이가 개구리 들어요 , predict:  0 (O)
0 0 코끼리 호랑이를 그려요
input:  코끼리 호랑이를 그려요 , predict:  1 (X)
0 0 다람쥐 원숭이를 던져요
input:  다람쥐 원숭이를 던져요 , predict:  1 (X)
0 0 다람쥐 꿀벌을 칠해요
input:  다람쥐 꿀벌을 칠해요 , predi

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.15
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.98
  Validation took: 0:00:03
0 0 호랑이가 코끼리를 그려요
input:  호랑이가 코끼리를 그려요 , predict:  0 (O)
0 0 원숭이가 다람쥐를 던져요
input:  원숭이가 다람쥐를 던져요 , predict:  0 (O)
0 0 꿀벌이 다람쥐를 칠해요
input:  꿀벌이 다람쥐를 칠해요 , predict:  0 (O)
0 0 꿀벌이 나비를 안아요
input:  꿀벌이 나비를 안아요 , predict:  0 (O)
0 0 다람쥐가 호랑이를 밀어요
input:  다람쥐가 호랑이를 밀어요 , predict:  0 (O)
0 0 개구리가 원숭이를 들어요
input:  개구리가 원숭이를 들어요 , predict:  0 (O)
0 0 코끼리가 호랑이 그려요
input:  코끼리가 호랑이 그려요 , predict:  0 (O)
0 0 다람쥐가 원숭이 던져요
input:  다람쥐가 원숭이 던져요 , predict:  0 (O)
0 0 다람쥐가 꿀벌 칠해요
input:  다람쥐가 꿀벌 칠해요 , predict:  0 (O)
0 0 나비가 꿀벌 안아요
input:  나비가 꿀벌 안아요 , predict:  0 (O)
0 0 호랑이가 다람쥐 밀어요
input:  호랑이가 다람쥐 밀어요 , predict:  0 (O)
0 0 원숭이가 개구리 들어요
input:  원숭이가 개구리 들어요 , predict:  0 (O)
0 0 코끼리 호랑이를 그려요
input:  코끼리 호랑이를 그려요 , predict:  1 (X)
0 0 다람쥐 원숭이를 던져요
input:  다람쥐 원숭이를 던져요 , predict:  1 (X)
0 0 다람쥐 꿀벌을 칠해요
input:  다람쥐 꿀벌을 칠해요 , predi

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.15
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.98
  Validation took: 0:00:03
0 0 호랑이가 코끼리를 그려요
input:  호랑이가 코끼리를 그려요 , predict:  0 (O)
0 0 원숭이가 다람쥐를 던져요
input:  원숭이가 다람쥐를 던져요 , predict:  0 (O)
0 0 꿀벌이 다람쥐를 칠해요
input:  꿀벌이 다람쥐를 칠해요 , predict:  0 (O)
0 0 꿀벌이 나비를 안아요
input:  꿀벌이 나비를 안아요 , predict:  0 (O)
0 0 다람쥐가 호랑이를 밀어요
input:  다람쥐가 호랑이를 밀어요 , predict:  0 (O)
0 0 개구리가 원숭이를 들어요
input:  개구리가 원숭이를 들어요 , predict:  0 (O)
0 0 코끼리가 호랑이 그려요
input:  코끼리가 호랑이 그려요 , predict:  0 (O)
0 0 다람쥐가 원숭이 던져요
input:  다람쥐가 원숭이 던져요 , predict:  0 (O)
0 0 다람쥐가 꿀벌 칠해요
input:  다람쥐가 꿀벌 칠해요 , predict:  0 (O)
0 0 나비가 꿀벌 안아요
input:  나비가 꿀벌 안아요 , predict:  0 (O)
0 0 호랑이가 다람쥐 밀어요
input:  호랑이가 다람쥐 밀어요 , predict:  0 (O)
0 0 원숭이가 개구리 들어요
input:  원숭이가 개구리 들어요 , predict:  0 (O)
0 0 코끼리 호랑이를 그려요
input:  코끼리 호랑이를 그려요 , predict:  1 (X)
0 0 다람쥐 원숭이를 던져요
input:  다람쥐 원숭이를 던져요 , predict:  1 (X)
0 0 다람쥐 꿀벌을 칠해요
input:  다람쥐 꿀벌을 칠해요 , predi